## Importing Every Unique Provider Specialty from all ACA databases

In [1]:
import pyodbc
import sys
import sqlalchemy
import urllib
import getpass
import re
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
HSA = ['Pareto_Capital_V2_TEMP','Pareto_Arizona','Pareto_Red_BCBS_Idaho','Pareto_WHA','Pareto_HarvardPilgrim','Pareto_BCBSKS_808']
WIP = ['Sharp_ParetoRed_2017Q4','KaiserWA_1093']
specialty = []

In [3]:
for i in WIP:
    params = urllib.parse.quote_plus('DSN=PW2WIPSQL01;DATABASE=%s;Trusted_Connection=yes' %i)
    engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
    result = engine.execute('SELECT DISTINCT providerSpecialtyDesc FROM A_004_Import_ProviderInfo')
    specialty.append(pd.DataFrame(result.fetchall()))

In [4]:
for i in HSA:
    params = urllib.parse.quote_plus('DSN=HSAAWSSQL03;DATABASE=%s;Trusted_Connection=yes' %i)
    engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
    result = engine.execute('SELECT DISTINCT providerSpecialtyDesc FROM A_004_Import_ProviderInfo')
    specialty.append(pd.DataFrame(result.fetchall()))

In [5]:
df_provider = pd.concat(specialty, ignore_index=True)
df_provider.columns = ['ProviderSpecialty']

In [6]:
df_provider['ProviderSpecialty2'] = df_provider.ProviderSpecialty.str.replace(r"([a-z])([A-Z])", r"\1 \2").str.strip()
df_provider['ProviderSpecialty2'] = df_provider.ProviderSpecialty2.str.lower().str.replace('/',' ').str.replace('-',' ')
df_provider['ProviderSpecialty2'] = df_provider.ProviderSpecialty2.str.replace('va provider',
                                                                        'veteran provider').str.replace('neuro', 'neuro ')
df_provider = df_provider[df_provider['ProviderSpecialty2']!='']

In [7]:
len(df_provider['ProviderSpecialty2'].sort_values().unique())

897

In [8]:
roots = ['acupunctur', 'anatom', 'anes', 'bariatric', 'behavior', 'chiro', 'derm', 
         'diab', 'diagnost','dialysis', 'diet', 'electro', 'endocrin', 'epilep', 'gastro', 
         'genet', 'ger', 'hand', 'head', 'hepatol','hospital',
         'hem', 'hyperbaric', 'infectious', 'maxillo', 'multi', 'naturo', 'neuro', 
         'nuclear', 'nutri', 'occupation', 'osteo', 'oto', 'pain', 'perfusion', 'pharma', 'plastic', 
         'podia', 'pulmon', 'radi', 'rehab', 'respira', 'rheum', 'sleep', 'speech', 'spin', 'surg', 
         'tele', 'temporomandibular', 'thera', 'thora', 'urgent', 'uro', 'vasc']
for root in roots:
    df_provider[root] = df_provider['ProviderSpecialty2'].str.contains(' '+root+'|^'+root)*1 

In [9]:
df_provider['ambulance']=df_provider['ProviderSpecialty2'].str.contains(' ambulance| paramedic|^ambulance|^paramedic')*1
df_provider['cardiology']=df_provider['ProviderSpecialty2'].str.contains(' cardi|^cardi| heart|^heart')*1
df_provider['clinic']=df_provider['ProviderSpecialty2'].str.contains(' clinic| care| center| ctr|^clinic|^care|^center|^ctr')*1
df_provider['colon']=df_provider['ProviderSpecialty2'].str.contains(' rectal| colo| proctol| endoscop|^rectal|^colo|^proctol|^endoscop')*1
df_provider['dental']=df_provider['ProviderSpecialty2'].str.contains('donti| dent| orthod|^dent|^orthod| oral|^oral')*1
df_provider['developdisable']=df_provider['ProviderSpecialty2'].str.contains(' autism| disab|^autism|^disab')*1
df_provider['emerg']=df_provider['ProviderSpecialty2'].str.contains(' emerg|^emerg| toxicology|^toxicology')*1
df_provider['fertility']=df_provider['ProviderSpecialty2'].str.contains('fertil|^family planning| reproduct|^reproduct')*1
df_provider['hearing']=df_provider['ProviderSpecialty2'].str.contains(' hearing| audio|^hearing|^audio')*1
df_provider['homeHealth']=df_provider['ProviderSpecialty2'].str.contains(' home|^home')*1
df_provider['hospice']=df_provider['ProviderSpecialty2'].str.contains(' hospice| assisted|^hospice|^assisted')*1
df_provider['imaging']=df_provider['ProviderSpecialty2'].str.contains(' mri|^mri|\(mri\)|\(pet\)|resonance| xray| ultrasound|^xray|^ultrasound')*1
df_provider['immune']=df_provider['ProviderSpecialty2'].str.contains(' immun| allerg|^immun|^allerg')*1
df_provider['internal']=df_provider['ProviderSpecialty2'].str.contains(' internal| int med|^internal|^int med')*1
df_provider['lab']=df_provider['ProviderSpecialty2'].str.contains(' blood| ^blood| microbio|^microbio| lab|^lab')*1
df_provider['medsupply']=df_provider['ProviderSpecialty2'].str.contains(' suppl| equip| prosth| orthop| orthot| shoe| wig|^orthot|^suppl|^equip|^prosth|^orthop|^shoe|^wig|component| foods|^foods')*1
df_provider['newborn']=df_provider['ProviderSpecialty2'].str.contains(' birthing| neonat| perinat| fetal| lactation| midwife|^birthing|^neonat|^perinat|^fetal|^lactation|^midwife')*1
df_provider['none']=df_provider['ProviderSpecialty2'].str.match('^n |^na$|null|none|ineligible provider|^not |^no |miscellaneous|atypical provider|other|unknown|error|missing')*1
df_provider['nurse']=df_provider['ProviderSpecialty2'].str.contains(' nurs| crn|^nurs|^crn')*1
df_provider['obgyn']=df_provider['ProviderSpecialty2'].str.contains('mammography| gyn| obstetrics| women|^gyn|^obstetrics|^women|^urogyn')*1
df_provider['oncology']=df_provider['ProviderSpecialty2'].str.contains(' oncolog| cancer|^oncolog|^cancer')*1
df_provider['ophthalmology']=df_provider['ProviderSpecialty2'].str.contains(' ophthalmology|^ophthalmology')*1
df_provider['optometry']=df_provider['ProviderSpecialty2'].str.contains(' opto|^opto')*1
df_provider['pathology']=df_provider['ProviderSpecialty2'].str.contains(' patho| cyto| toxicology|^patho|^cyto|^toxicology|pathology$')*1
df_provider['PCP']=df_provider['ProviderSpecialty2'].str.contains(' physician|^physician| practice|^practice| pcp|^pcp| prac|^prac| preventitive|^preventitive| intervention|^intervention| preventive|^preventive| family medicine|^family medicine| preventative|^preventative')*1
df_provider['pediatrics']=df_provider['ProviderSpecialty2'].str.contains(' pedi| pedo| child| adolescent|^adolescent|^pedi|^pedo|^child| peds|^peds')*1
df_provider['physicalTherapy']=df_provider['ProviderSpecialty2'].str.contains(' physical|^physical| physia|^physia| sport|^sport| manipulative|^manipulative')*1
df_provider['psychiatry']=df_provider['ProviderSpecialty2'].str.contains(' psychiatry|^psychiatry| psychiatric|^psychiatric')*1
df_provider['psychology']=df_provider['ProviderSpecialty2'].str.contains(' psychology| social| counsel| mental|^psychology|^social|^counsel|^mental|^eating disorder| psycholog|^psycholog| marriage|^marriage')*1
df_provider['publicHealth']=df_provider['ProviderSpecialty2'].str.contains(' public| county| local| community| charit| volun|^public|^county|^local|^community|^charit|^volun| educat|^educat')*1
df_provider['renal']=df_provider['ProviderSpecialty2'].str.contains(' nephro| litho|^nephro|^litho')*1
df_provider['substance']=df_provider['ProviderSpecialty2'].str.contains(' substance| addi|^substance|^addi| depend|^depend| alcohol|^alcohol')*1
df_provider['residental']=df_provider['ProviderSpecialty2'].str.contains(' residental|^residental')*1
df_provider['trans']=df_provider['ProviderSpecialty2'].str.contains(' transplant|^transplant')*1
df_provider['veteran']=df_provider['ProviderSpecialty2'].str.contains(' veteran|^veteran| military|^military| defense|^defense| aerospace|^aerospace')*1

In [10]:
df_provider[df_provider.columns[1:]]

,ProviderSpecialty2,acupunctur,ambul,anatom,anes,bariatric,behavior,chiro,derm,diab,...,pathology,PCP,pediatrics,physicalTherapy,psychiatry,psychology,publicHealth,substance,trans,veteran
0,"internal medicine, endocrinology, diabetes & m...",0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,not specified,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,internal medicine,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,pediatrics,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,"internal medicine, nephrology",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"dietitian, registered",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,"emergency medicine, sports medicine",0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
7,obstetrics & gynecology,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,family medicine,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,multi specialty,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
df_provider=df_provider.append(df_provider.sum(numeric_only=True), ignore_index=True)

In [12]:
#df_provider.to_csv(r'/mnt/ds/notebooks/NathanStevenson/df_provider.csv')

In [13]:
df_provider[df_provider["nutri"]==1]

,ProviderSpecialty,ProviderSpecialty2,acupunctur,ambul,anatom,anes,bariatric,behavior,chiro,derm,...,pathology,PCP,pediatrics,physicalTherapy,psychiatry,psychology,publicHealth,substance,trans,veteran
78,NUTRITION,nutrition,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
194,Licensed Dietician/Nutritionis,licensed dietician nutritionis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
228,Nutritionist,nutritionist,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
465,NUTRITIONIST,nutritionist,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
720,Nutritionist,nutritionist,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
787,Dietician Reg Nutrition Metabo,dietician reg nutrition metabo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
920,Nutritionist,nutritionist,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1110,Nutrition,nutrition,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1246,NUTRITIONIST ...,nutritionist,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
df_provider.head()

,ProviderSpecialty,ProviderSpecialty2,acupunctur,ambul,anatom,anes,bariatric,behavior,chiro,derm,...,pathology,PCP,pediatrics,physicalTherapy,psychiatry,psychology,publicHealth,substance,trans,veteran
0,"Internal Medicine, Endocrinology, Diabetes & M...","internal medicine, endocrinology, diabetes & m...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Not Specified,not specified,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Internal Medicine,internal medicine,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Pediatrics,pediatrics,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Internal Medicine, Nephrology","internal medicine, nephrology",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
df_provider.columns[2:89]

Index(['acupunctur', 'ambul', 'anatom', 'anes', 'bariatric', 'behavior',
       'chiro', 'derm', 'diab', 'diagnost', 'dialysis', 'diet', 'electro',
       'endocrin', 'epilep', 'gastro', 'genet', 'ger', 'hand', 'head',
       'hepatol', 'hospital', 'hem', 'hyperbaric', 'infectious', 'maxillo',
       'multi', 'naturo', 'neuro', 'nuclear', 'nutri', 'occupation', 'osteo',
       'oto', 'pain', 'perfusion', 'pharma', 'plastic', 'podia', 'pulmon',
       'radi', 'rehab', 'respira', 'rheum', 'sleep', 'speech', 'spin', 'surg',
       'tele', 'temporomandibular', 'thera', 'thora', 'urgent', 'uro', 'vasc',
       'cardiology', 'clinic', 'colon', 'dental', 'developdisable', 'emerg',
       'fertility', 'hearing', 'hospice', 'imaging', 'immune', 'internal',
       'kidney', 'lab', 'medsupply', 'newborn', 'none', 'nurse', 'obgyn',
       'oncology', 'ophthalmology', 'optometry', 'pathology', 'PCP',
       'pediatrics', 'physicalTherapy', 'psychiatry', 'psychology',
       'publicHealth', 'substan

In [17]:
df_provider.columns[2]

'acupunctur'

In [18]:
str(df_provider.columns[2])+' '

'acupunctur '

In [19]:
range = np.arange(2,89)

In [20]:
range

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
       70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86,
       87, 88])

In [21]:
del df_test

NameError: name 'df_test' is not defined

In [22]:
df_test=df_provider

In [23]:
for root in range:
    df_test[df_test.columns[root]].replace({0:'',1:str(df_test.columns[root])+','},inplace=True)

In [25]:
cols = df_provider.columns[2:90]

In [26]:
cols

Index(['acupunctur', 'ambul', 'anatom', 'anes', 'bariatric', 'behavior',
       'chiro', 'derm', 'diab', 'diagnost', 'dialysis', 'diet', 'electro',
       'endocrin', 'epilep', 'gastro', 'genet', 'ger', 'hand', 'head',
       'hepatol', 'hospital', 'hem', 'hyperbaric', 'infectious', 'maxillo',
       'multi', 'naturo', 'neuro', 'nuclear', 'nutri', 'occupation', 'osteo',
       'oto', 'pain', 'perfusion', 'pharma', 'plastic', 'podia', 'pulmon',
       'radi', 'rehab', 'respira', 'rheum', 'sleep', 'speech', 'spin', 'surg',
       'tele', 'temporomandibular', 'thera', 'thora', 'urgent', 'uro', 'vasc',
       'cardiology', 'clinic', 'colon', 'dental', 'developdisable', 'emerg',
       'fertility', 'hearing', 'hospice', 'imaging', 'immune', 'internal',
       'kidney', 'lab', 'medsupply', 'newborn', 'none', 'nurse', 'obgyn',
       'oncology', 'ophthalmology', 'optometry', 'pathology', 'PCP',
       'pediatrics', 'physicalTherapy', 'psychiatry', 'psychology',
       'publicHealth', 'substan

In [27]:
df_test['root_name'] = df_test[cols].apply(lambda row: ''.join(row.values.astype(str)), axis=1)

In [28]:
df_test.head()

,ProviderSpecialty,ProviderSpecialty2,acupunctur,ambul,anatom,anes,bariatric,behavior,chiro,derm,...,PCP,pediatrics,physicalTherapy,psychiatry,psychology,publicHealth,substance,trans,veteran,root_name
0,"Internal Medicine, Endocrinology, Diabetes & M...","internal medicine, endocrinology, diabetes & m...",,,,,,,,,...,,,,,,,,,,"diab,endocrin,internal,"
1,Not Specified,not specified,,,,,,,,,...,,,,,,,,,,"none,"
2,Internal Medicine,internal medicine,,,,,,,,,...,,,,,,,,,,"internal,"
3,Pediatrics,pediatrics,,,,,,,,,...,,"pediatrics,",,,,,,,,"pediatrics,"
4,"Internal Medicine, Nephrology","internal medicine, nephrology",,,,,,,,,...,,,,,,,,,,"internal,kidney,"


In [29]:
df_test.to_csv(r'/mnt/ds/notebooks/NathanStevenson/df_provider_combo.csv')

In [75]:
df_prov_ref = pd.read_csv('df_provider_combo.csv')

In [76]:
df_prov_ref.head()

,ProviderSpecialty,ProviderSpecialty2,root_name,category1,category2
0,AUDIOLOGY,audiology,"hearing,",Audiology,NaN
1,Audiology,audiology,"hearing,",Audiology,NaN
2,Hearing Aid Fitter/Dealer,hearing aid fitter dealer,"hearing,",Audiology,NaN
3,AUDIOLOGIST,audiologist,"hearing,",Audiology,NaN
4,HEARING AID,hearing aid,"hearing,",Audiology,NaN
